In [ ]:
from pymongo import MongoClient
import pandas as pd
import pymongo
from pandas.io.json import json_normalize
import re
import numpy as np


In [2]:
# #Connecting the database with the queried data (data_companies_clean)
# client = MongoClient('mongodb://localhost:27017/')
# db = client.companies
# data = db.data_companies_clean


In [3]:
#Connecting the database with the queried data (data_companies_clean)
client = MongoClient('mongodb://localhost:27017/')
db = client.DBcompanies_cb
data = db.companies_cb

In [4]:
#Query to receive all the essential data for my analysis. 
one_office = db.companies_cb.find({'$and':[
    {'offices':{'$exists':True}},
    {'offices':{'$ne': None}},
#     {'number_of_employees':{'$exists':True}}
    {'number_of_employees':{'$gte': 10}}
]})


In [5]:
#Create first dataframe and show the columns we have
one_office = pd.DataFrame(one_office)
one_office.columns

Index(['_id', 'acquisition', 'acquisitions', 'alias_list', 'blog_feed_url',
       'blog_url', 'category_code', 'competitions', 'created_at',
       'crunchbase_url', 'deadpooled_day', 'deadpooled_month',
       'deadpooled_url', 'deadpooled_year', 'description', 'email_address',
       'external_links', 'founded_day', 'founded_month', 'founded_year',
       'funding_rounds', 'homepage_url', 'image', 'investments', 'ipo',
       'milestones', 'name', 'number_of_employees', 'offices', 'overview',
       'partners', 'permalink', 'phone_number', 'products', 'providerships',
       'relationships', 'screenshots', 'tag_list', 'total_money_raised',
       'twitter_username', 'updated_at', 'video_embeds'],
      dtype='object')

In [6]:
#Merge deadpool related columns into 1 and fill blanks with NaN values.
one_office['deadpooled'] = one_office[one_office.columns[10:13]].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1).replace(r'^\s*$', np.nan, regex=True)


In [7]:
#Select relevant columns for the project
data = pd.DataFrame(one_office[['name', 'category_code', 'founded_year', 'number_of_employees', 'offices','total_money_raised', 'deadpooled']])


In [8]:
#Select alive companies. If they have 'deadpoled' data I understand they are dead. 
data = data[pd.isnull(data['deadpooled'])]
data.head()


,name,category_code,founded_year,number_of_employees,offices,total_money_raised,deadpooled
0,Wetpaint,web,2005.0,47,"[{'description': '', 'address1': '710 - 2nd Av...",$39.8M,NaN
1,AdventNet,enterprise,1996.0,600,"[{'description': 'Headquarters', 'address1': '...",$0,NaN
2,Zoho,software,2005.0,1600,"[{'description': 'Headquarters', 'address1': '...",$0,NaN
3,Digg,news,2004.0,60,"[{'description': None, 'address1': '135 Missis...",$45M,NaN
4,Facebook,social,2004.0,5299,"[{'description': 'Headquarters', 'address1': '...",$2.43B,NaN


In [9]:
#Inspect info provided inside offices column
c = pd.DataFrame(json_normalize(data['offices'][4]))
c


,address1,address2,city,country_code,description,latitude,longitude,state_code,zip_code
0,1601 Willow Road,,Menlo Park,USA,Headquarters,37.416050,-122.151801,CA,94025
1,,,Dublin,IRL,Europe HQ,53.344104,-6.267494,None,
2,340 Madison Ave,,New York,USA,New York,40.755716,-73.979247,NY,10017


In [10]:
data2 = data.copy()
len(data2)

4406

In [11]:

office_split = pd.DataFrame(data2['offices'].tolist()).stack().reset_index(level=1, drop=True).rename('office')
office_merged = data2.merge(office_split, left_index=True, right_index=True)


In [12]:
len(office_merged)

5856

In [13]:
office_merged = office_merged.reset_index()
office_merged.head()

,index,name,category_code,founded_year,number_of_employees,offices,total_money_raised,deadpooled,office
0,0,Wetpaint,web,2005.0,47,"[{'description': '', 'address1': '710 - 2nd Av...",$39.8M,NaN,"{'description': '', 'address1': '710 - 2nd Ave..."
1,0,Wetpaint,web,2005.0,47,"[{'description': '', 'address1': '710 - 2nd Av...",$39.8M,NaN,"{'description': '', 'address1': '270 Lafayette..."
2,1,AdventNet,enterprise,1996.0,600,"[{'description': 'Headquarters', 'address1': '...",$0,NaN,"{'description': 'Headquarters', 'address1': '4..."
3,2,Zoho,software,2005.0,1600,"[{'description': 'Headquarters', 'address1': '...",$0,NaN,"{'description': 'Headquarters', 'address1': '4..."
4,3,Digg,news,2004.0,60,"[{'description': None, 'address1': '135 Missis...",$45M,NaN,"{'description': None, 'address1': '135 Mississ..."


In [ ]:
#Function to convert the info within offices into columns.
def latlong(data):
    data = data['offices']
    principal = None
    if data[0]['latitude'] and data[0]['longitude']:                   #Check there is data
        principal = {
            "type":"Point",
            "coordinates":[data[0]['longitude'], data[0]['latitude']]
        }

    return {
        "totalOffices": len(data),
        "lat": data[0]['latitude'],
        "lng": data[0]['longitude'],
        "main_office (geoquery)": principal
    }

data_latlong = data[["offices"]].apply(latlong, result_type="expand", axis=1).dropna()


In [ ]:
#Concatenate all data
data = pd.concat([data, data_latlong], axis = 1)


In [ ]:
data = data[[x for x in data.columns if x !='deadpooled' and x != 'offices']]

In [ ]:
data.head()

In [ ]:
a = json_normalize(data['main_office (geoquery)'][0])
a

In [ ]:
data.dtypes

In [ ]:
data.fillna(0).head()

In [ ]:
data['founded_year'].astype(int)

In [ ]:
def fixints(col):
    data[col].fillna(0, inplace = True)
    data[col] = data[col].astype(int)
    return data[col]

data = fixints ('founded_year')

In [ ]:
# data_cols = data_companies_1office[['alias_list', 'category_code','description', 'total_money_raised','founded_year','offices', 'deadpooled_year', 'deadpooled_day', 'deadpooled_month',
#        'deadpooled_url' ]]

In [ ]:
def latlong(data):
    data = data['offices']
#    return (len(data),data[0]['latitude'],data[0]['longitude'])

    # Only create the geoJSON object if all geodata is available
    principal = None
    if data[0]['latitude'] and data[0]['longitude']:
        principal = {
            "type":"Point",
            "coordinates":[data[0]['longitude'], data[0]['latitude']]
        }

    return {
        "totalOffices": len(data),
        "lat": data[0]['latitude'],
        "lng": data[0]['longitude'],
        "main_office": principal
    }


data_latlong = data_cols[["offices"]].apply(latlong, result_type="expand", axis=1)

In [ ]:
data_latlong.dropna(inplace = True)

In [ ]:
display(data_latlong.head())


In [ ]:
# def latlong(df):
#     df = df['offices']
#     for l in df:
#         if l['latitude'] and l['longitude']:
#             return {
#                     "lat": l['latitude'],
#                     "long": l['longitude']
                    
#                 }

# first_office = data_cols[['offices']].apply(latlong, result_type = "expand", axis=1)
# display(first_office.head())

In [ ]:
data_office = pd.concat([data_cols, data_latlong],axis=1)
display(data_office.head())
data_office.shape

In [ ]:
data_office.dropna(subset=['lat','lng'], inplace = True)
data_office.shape


In [ ]:
# data_companies = data_latlong[['alias_list', 'category_code', 'description', 'total_money_raised', 'founded_year', 'deadpooled_year', 'deadpooled_day', 'deadpooled_month',
#        'deadpooled_url', 'lat', 'long']]
# data_companies.shape

In [ ]:
#drop notnull values in deadpool = if it has a value it means the startup is dead.
data2 = data_office[pd.isnull(data_office['deadpooled_year'])]
data3 = data2[pd.isnull(data_office['deadpooled_day'])]
data4 = data3[pd.isnull(data_office['deadpooled_month'])]
data_deads = data4[pd.isnull(data_office['deadpooled_url'])]


data2.shape

In [ ]:
display(data_deads.head())

In [ ]:
data_companies_clean = data_deads[['alias_list', 'category_code', 'description', 'total_money_raised', 'founded_year', 'main_office', 'lat','lng']]
display(data_companies_clean.head())
data_companies_clean.shape

In [ ]:
data_year = data_companies_clean[data_companies_clean['founded_year']>2006]
data_year.shape

In [ ]:
data_web = data_year[data_year['category_code'] == 'web']
data_web.shape

In [ ]:
data_final = data_web[data_web['total_money_raised'] != '$0']
data_final.shape

In [ ]:
#The idea is to develop elaborated backend products to be sold to other web startup. other startups. Because of that, I only select young startups
#which have been funded and are not big enough to develop these products by their own. 

In [ ]:
display(data_final.head())

In [ ]:
data_final['main_office'][0]

In [ ]:
data_final.to_json('/Users/alejandroiborralucas/Desktop/Git-iron/Project crunchbase/visualizing-real-world-data-project/oficinas.json', orient="records")

In [ ]:
# def splitMoney(df): 
#     x = re.findall('([^\d+])', df)
#     return x
# data_companies_clean['Currency type'] = data_companies_clean['total_money_raised'].apply(splitMoney)

# #display(data_companies_clean['Currency type'].head())
# display(data_companies_clean.head())

In [ ]:
# data_companies_clean['Currency type'].value_counts()

In [ ]:
# values=[]
# a = ['$', '€', '£', 'C', 'k']
# for i in data_companies_clean['Currency type']:
#     if i[0] not in values:
#         values.append(i[0])
# print(values)

In [ ]:
# #VOY POR AQUI. MONTAR FUNCION USANDO REGEX QUE BUSQUE M Y MULTIPLIQUE POR 1X10^6, ETC. + API PARA TIPOS DE CAMBIO

# def valueMultiplier(df):
#     x = re.findall('(\d+)', df)
    
#     return x

# data_companies_clean['Value Gross'] = data_companies_clean['total_money_raised'].apply(valueMultiplier)
    

In [ ]:
#def findCurrency (m):
#    currency_dict = {
#        '$': 1
#        '€': 1.12
#        '£': 1.25
#        'C$': 0.76
#        'kr': 0.11
#    }
#for currency, value in findMonth.items():
#    if currency in m:
#        return int(value)
#    return 0 
#data_companies_clean['Currency type'] = data_companies_clean['total_money_raised'].apply(findCurrency)

In [ ]:
# data_companies_clean['a'] = [data_companies_clean['total_money_raised'].str.split('$', expand=True)]

# display(data_companies_clean.head())

In [ ]:
# def classifyCurrencies(df):
#     money_type = []
    
#     for i in splitCurrencies(df):
#         if i in currencies:
#             money_type.append(i)
#     return money_type

In [ ]:
# #tipos de monedas en el dataset
# currencies_types=[]
# for i in data_companies_clean['total_money_raised']:
#     if i[0:2] not in currencies_types:
#         currencies_types.append(i[0:2])

# print(currencies_types)

In [ ]:
# currency=[]
# for i in data_companies_clean['total_money_raised']:
#     currency.append(i[0:2])



In [ ]:
# currency_type = pd.DataFrame({'Currency':currency})
# display(currency_type.head())

In [ ]:
# def splitCurrencies(df):
#     split = df.str.split('')
#     return split




In [ ]:
# def classifyCurrencies(df):
#     money_type = []
#     for i in splitCurrencies(df):
#         if i in currencies:
#             money_type.append(i)
#     return money_type

# classifyCurrencies(data_companies_clean[['total_money_raised']])

In [ ]:
# data_companies_clean['currency']= data_companies_clean[['total_money_raised']].apply(splitCurrencies, result_type = 'expand', axis=1)


In [ ]:
# display(data_companies_clean.head())